# Profiling
Sometimes computing the likelihood is not as fast as we would like. Theano provides handy profiling tools which are wrapped in PyMC3 by `model.profile`. This function returns a `ProfileStats` object conveying information about the underlying Theano operations. Here we'll profile the likelihood and gradient for the stochastic volatility example.

First we build the model.

In [1]:
import pandas as pd
from pymc3 import *
from pymc3.math import exp
from pymc3.distributions.timeseries import *

returns = pd.read_csv(get_data('SP500.csv'), index_col=0, parse_dates=True)

n = returns.shape[0]

with Model() as model:
    sigma = Exponential('sigma', 1. / .02, testval=.1)
    nu = Exponential('nu', 1. / 10)
    s = GaussianRandomWalk('s', sigma ** -2, shape=n)
    r = StudentT('r', nu, lam=exp(-2 * s), observed=returns)

Then we call the `profile` function and summarize its return values.

In [2]:
# Profiling of the logp call
model.profile(model.logpt).summary()

Function profiling
  Message: /home/junpenglao/Documents/pymc3/pymc3/model.py:921
  Time in 1000 calls to Function.__call__: 2.225540e+00s
  Time in Function.fn.__call__: 2.201248e+00s (98.908%)
  Time in thunks: 2.190614e+00s (98.431%)
  Total compile time: 7.304576e+00s
    Number of Apply nodes: 26
    Theano Optimizer time: 5.187931e-01s
       Theano validate time: 1.646996e-03s
    Theano Linker time (includes C, CUDA code generation/compiling): 6.748426e+00s
       Import time 3.453946e-02s
       Node make_thunk time 6.747270e+00s
           Node Elemwise{add,no_inplace}(TensorConstant{(1, 1) of 1.0}, InplaceDimShuffle{x,x}.0) time 8.496084e-01s
           Node Elemwise{Composite{((i0 + (i1 * log(((i2 * i3) / i4)))) - i5)}}(Elemwise{Composite{scalar_gammaln((i0 * i1))}}.0, TensorConstant{(1, 1) of 0.5}, TensorConstant{(1, 1) of ..8861837907}, Elemwise{Composite{exp((i0 * i1))}}.0, InplaceDimShuffle{x,x}.0, Elemwise{Composite{scalar_gammaln((i0 * i1))}}.0) time 7.832110e-01s
   

In [3]:
# Profiling of the gradient call dlogp/dx
model.profile(gradient(model.logpt, model.vars)).summary()

Function profiling
  Message: /home/junpenglao/Documents/pymc3/pymc3/model.py:921
  Time in 1000 calls to Function.__call__: 4.595304e+00s
  Time in Function.fn.__call__: 4.530428e+00s (98.588%)
  Time in thunks: 4.328702e+00s (94.198%)
  Total compile time: 1.276115e+01s
    Number of Apply nodes: 49
    Theano Optimizer time: 9.874594e-01s
       Theano validate time: 5.513906e-03s
    Theano Linker time (includes C, CUDA code generation/compiling): 1.172469e+01s
       Import time 5.937672e-02s
       Node make_thunk time 1.172257e+01s
           Node Elemwise{Composite{exp((i0 * i1))}}(TensorConstant{(1,) of -2.0}, s) time 1.043019e+00s
           Node Elemwise{Composite{Switch(i0, (-log1p((i1 / i2))), i3)}}(Elemwise{Composite{Cast{int8}((GT(i0, i1) * i2 * GT(inv(sqrt(i0)), i1)))}}.0, Elemwise{mul,no_inplace}.0, InplaceDimShuffle{x,x}.0, TensorConstant{(1, 1) of 0}) time 9.094841e-01s
           Node Elemwise{mul,no_inplace}(TensorConstant{(1, 1) of -0.5}, Elemwise{add,no_inplace}.

In [47]:
%load_ext watermark
%watermark -n -u -v -iv -w

pymc3   3.8
arviz   0.7.0
pandas  0.25.3
seaborn 0.9.0
numpy   1.17.5
last updated: Wed Apr 22 2020 

CPython 3.8.0
IPython 7.11.0
watermark 2.0.2
